# POSEIDON: Pose Estimation & Activity Recognition using GNNs

Team Members (Group 16): 
1. Chong Jun Rong Brian (A0290882U)
2. Parashara Ramesh (A0285647M)
3. Ng Wei Jie Brandon (A0184893L)

In [ ]:
%load_ext autoreload
%autoreload 2

<h2><u> Table of contents </u></h2>

1. What is this project about?
<br> 1.1. Project Motivation
<br> 1.2. Project Description
<br> 1.3. Project Setup   
2. Human 3.6M Dataset
<br> 2.1. Summary of the dataset
<br> 2.2. Preparing the dataset
<br> 2.3. Visualizing poses
3. Model building approach
4. Baseline 1 - SimplePose (Simple ML model without using GNNs)
5. Baseline 2 - SimplePoseGNN (Simple ML model using GNNs) 
6. Improvement 1 - SimplePoseGAT (Using 2 layers of GATConv)
7. Improvement 2 - SemGCN model (Reimplementation of Semantic GCN)
8. Evaluation & Analysis of models
9. Creating our own custom dataset
10. Evaluation on custom dataset
11. Conclusion
12. Video presentation & Resources


<h2><u>1. What is this project about?</u></h2>
<h3><u>1.1 Project Motivation</u></h3>

Accurately predicting 3D human poses from 2D keypoints is a critical task for many applications such as motion capture and activity recognition. Traditional methods that use direct regression or lifting techniques often struggle to fully capture the complex spatial relationships between body joints. By treating the 2D pose keypoints as graphs, we can leverage the underlying connectivity between joints to improve the 3D pose estimation. Additionally, recognizing and classifying human activities from these poses is an essential task in fields like surveillance and healthcare. Therefore, this project seeks to explore how GNNs can enhance 3D pose estimation and activity recognition.

<h3><u>1.2 Project Description</h3></u>

The primary objective of this project is to predict 3D human poses from 2D pose keypoints accurately using GNNs. 
* Firstly, we will develop two baseline models: one using standard Neural Network (NN) & Convolutional Neural Network (CNN) followed by a simple GNN based model both for 3d pose estimation 
* Secondly, we will reimplement the SemGCN model, which treats the body joints of a 2D pose as nodes in a graph, with edges representing the connectivity between them. 
* Finally, we will design an improved version of the SemGCN model by exploring different GNN architectures and modifications to enhance its performance.

The secondary objective is to classify human activities based on 2D pose keypoints. We will use custom datasets to validate this task, allowing us to assess the generalization capabilities of GNN-based models for activity recognition.

<h3><u>1.3 Project Setup</u></h3>

1. Install the dependencies from requirements.txt using the command

`pip install -r requirements.txt`


<h2><u>2. Human 3.6M Dataset</u></h2>

<h3><u>2.1 Summary of the dataset </u></h3>

The [Human 3.6M dataset](http://vision.imar.ro/human3.6m/description.php) is a large-scale collection of 3.6 million 3D human poses captured from 11 professional actors in 17 everyday scenarios (e.g., talking, smoking, discussing). It includes synchronized video from multiple calibrated cameras, precise 3D joint positions and angles, and pixel-level body part labels. Additional data like time-of-flight range data and 3D laser scans of actors are also available.

The dataset comes with precomputed image descriptors, tools for visualizing and predicting human poses, and an evaluation set for benchmarking performance, making it a rich resource for 3D human pose estimation, action recognition, and related computer vision tasks.

<h3><u>2.2. Dataset preparation </u></h3>

Downloading the dataset from the website requires a login , therefore we will be directly using a preprocessed version of this dataset stored in google drive [here](https://drive.google.com/file/d/1JGt3j9q5A8WzUY-QKfyMVJUUvfreri-s/view?usp=drive_link).
 
Original preprocessing was done by [Martinez et al](https://github.com/una-dinosauria/3d-pose-baseline) from this repository.

The cells below go through the steps of downloading this preprocessed dataset and creating train-test files from it after some transformation. 

From section 3(Models) we will be directly using the created train-test files, therefore the cells under this section need not be executed.

#### Downloading the zip from google drive

In [ ]:
# Download the zip from google drive
import gdown
import os

# file id from gdrive (refer to markdown cell above for full link)
file_id = '1JGt3j9q5A8WzUY-QKfyMVJUUvfreri-s' 
download_url = f'https://drive.google.com/uc?id={file_id}'

# Folder where you want to save the file
folder_path = os.path.join(os.getcwd(),"datasets", "h36m", "Original")
zip_file_path = os.path.join(folder_path, 'h36m.zip')

# Ensure the folder exists
os.makedirs(folder_path, exist_ok=True)

# Download the ZIP file
gdown.download(download_url, zip_file_path, quiet=False)

print(f"Downloaded ZIP file and saved to {zip_file_path}")


#### Extracting the contents into the same folder

In [ ]:
# extracting the contents inside the /datasets/h36m/Original folder
import zipfile

print(f"Extracting files to {folder_path}..")
# Open the ZIP file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(folder_path)

print(f"Finished extracting files to {folder_path}")



#### Saving the 3d positions into a compressed np file

In [ ]:
import h5py
from glob import glob
from tqdm import tqdm
import numpy as np

output_filename = os.path.join(folder_path, 'data_3d_h36m')
subjects = ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11']

output = dict()
for subject in tqdm(subjects, desc= f'Processing subjects..'):
    output[subject] = dict()
    file_list = glob(os.path.join(folder_path, 'h36m', subject, 'MyPoses', '3D_positions', '*.h5'))
    assert len(file_list) == 30, "Expected 30 files for subject " + subject + ", got " + str(len(file_list))
    for f in file_list:
        action = os.path.splitext(os.path.basename(f))[0]

        if subject == 'S11' and action == 'Directions':
            continue  # Discard corrupted video

        with h5py.File(f) as hf:
            positions = hf.get('3D_positions')[:].reshape(32, 3, -1).transpose(2, 0, 1)
            positions /= 1000  # Meters instead of millimeters
            output[subject][action] = positions.astype('float32')

print(f'Saving compressed 3d positions into {output_filename}')
np.savez_compressed(output_filename, positions_3d=output)
print(f'Finished saving 3d positions into {output_filename}')
del output


#### Compute the ground truth 2d poses.

TODO.brandon explain what the human35mDataset constructor does in detail (intrinsics, extrinsics, cameras, normalization etc etc)

In [ ]:
from data.camera import world_to_camera, project_to_2d, image_coordinates, wrap
from data.h36m_dataset import Human36mDataset

print('Computing ground-truth 2D poses...')
output_filename_2d = os.path.join(folder_path, 'data_2d_h36m_gt')

dataset = Human36mDataset(output_filename + '.npz')
output_2d_poses = {}
for subject in dataset.subjects():
    output_2d_poses[subject] = {}
    for action in dataset[subject].keys():
        anim = dataset[subject][action]

        positions_2d = []
        for cam in anim['cameras']:
            pos_3d = world_to_camera(anim['positions'], R=cam['orientation'], t=cam['translation'])
            pos_2d = wrap(project_to_2d, True, pos_3d, cam['intrinsic'])
            pos_2d_pixel_space = image_coordinates(pos_2d, w=cam['res_w'], h=cam['res_h'])
            positions_2d.append(pos_2d_pixel_space.astype('float32'))
        output_2d_poses[subject][action] = positions_2d

print(f'Saving compressed 2d positions into {output_filename_2d}')
metadata = {
    'num_joints': dataset.skeleton().num_joints(),
    'keypoints_symmetry': [dataset.skeleton().joints_left(), dataset.skeleton().joints_right()]
}
np.savez_compressed(output_filename_2d, positions_2d=output_2d_poses, metadata=metadata)

print(f'Done saving compressed 2d positions into {output_filename_2d}')
del output_2d_poses

#### Cleanup the extracted files and the downloaded zip

In [ ]:
from shutil import rmtree

rmtree(os.path.join(folder_path, 'h36m'))
os.remove(os.path.join(folder_path, 'h36m.zip'))

#### Creating and saving train and test files from the saved npz files

In [ ]:
from utils.data_utils import read_3d_data, create_2d_data, create_train_test_files
from data.h36m_dataset import TRAIN_SUBJECTS, TEST_SUBJECTS, Human36mDataset
import os

subjects_train = TRAIN_SUBJECTS
subjects_test = TEST_SUBJECTS

processed_dataset_path = os.path.join(os.getcwd(), "datasets", "h36m", "Processed")
os.makedirs(processed_dataset_path, exist_ok=True)

dataset_path = os.path.join(os.getcwd(), "datasets", "h36m", "Original", "data_3d_h36m.npz")
dataset = Human36mDataset(dataset_path)


In [ ]:
#TODO.brandon: add comments for what exactly the read_3d_data function does and the create_2d_data does.
print("Reading 3d npz file")
dataset = read_3d_data(dataset)

print("Reading 2d npz file")
dataset_2d_path = os.path.join(os.getcwd(), "datasets", "h36m", "Original", "data_2d_h36m_gt.npz")
keypoints = create_2d_data(dataset_2d_path, dataset)
print("Done")

In [ ]:
# NOTE: train on subset of distinct actions or on full set of actions
keep_actions = ['Greeting', 'Photo', 'Sitting', 'SittingDown', 'Walking']
keep_actions = []

print("Creating train datasets and saving into file")
_, _, _ = create_train_test_files(subjects_train, dataset, keypoints, "train", processed_dataset_path, keep_actions=keep_actions)

print("Creating test datasets and saving into file")
_, _, _ = create_train_test_files(subjects_test, dataset, keypoints, "test", processed_dataset_path, keep_actions=keep_actions)
print("Done")

<h3><u>2.3 Visualizing poses </u></h3>

Since we have saved the train and test files, we will now only use those files for the rest of the notebook.

This has also been saved directly in this [google drive link](https://drive.google.com/drive/folders/1d38hWSM8clZlI11nqljxBECog1KY-iTd?usp=sharing), which can be placed directly in the local path instead of executing previous cells.

The visualizations below shows that we are considering that human poses are comprised of skeletons with just 16 joints after doing all of the processing above.

In [ ]:
import numpy as np

from utils.visualization_utils import visualize_2d_pose, visualize_3d_pose

poses_2d = np.load("datasets/h36m/Processed/test_2d_poses.npy")
poses_3d = np.load("datasets/h36m/Processed/test_3d_poses.npy")

visualize_2d_pose(poses_2d[0])
visualize_3d_pose(poses_3d[0], elev=110, azim=90)

#### Finding out the distribution of actions in the train and test splits

The following code cells plots a bar graph of the actions in the train and test datasets.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def show_action_stats(actions, type):
    train_actions_df = pd.DataFrame(actions, columns=['actions'])
    action_counts = train_actions_df['actions'].value_counts()
    print(action_counts)
    
    action_counts.plot(kind='bar')
    
    # Step 4: Show the plot
    plt.title(f'{type} Action Distribution')
    plt.xlabel('Action')
    plt.ylabel('Count')
    plt.show()


In [ ]:
train_actions = np.load("datasets/h36m/Processed/train_actions.npy")
show_action_stats(train_actions, "Train")

In [ ]:
import torch 
import torch.nn as nn

w = nn.Parameter(torch.Tensor(size=(2,5,5)))

In [ ]:
test_actions = np.load("datasets/h36m/Processed/test_actions.npy")
show_action_stats(train_actions, "Test")

<h2><u>3. Model Building Approach</u></h2>

The following sections attempts to build different ML models using the dataset which was just constructed above.

At a very high level these are the models we aim to build:
* SimplePose - A simple baseline ML model using classic DL layers
* SimplePoseGNN - A simple baseline GNN model using DGL and GNN layers
* SemGCN - A reimplementation of the [SemGCN paper](https://arxiv.org/abs/1904.03345) in DGL
* PoseGCN - A more complex GNN model by leveraging the insights gained from building the other GNN models.


<h2><u>4. Baseline 1 - SimplePose (Simple ML model without using GNNs)</u></h2>

TODO.brian - write something meaningful here



In [ ]:
from src.simple_pose.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 0.005,
    'pose_loss_multiplier': 0.995,
    'training_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'custom', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

INFO:root:Model is currently using : cuda
INFO:root:Creating Missing Paths: model_outputs\simple_pose\2024-11-13--16-12-40
INFO:root:Saving config
INFO:root:Setup Training and Testing Dataloaders
INFO:root:Setup SimplePose model with Kaiming Weights
INFO:root:SimplePose(
  (input_linear_2d): Linear(in_features=26, out_features=1024, bias=True)
  (blocks): ModuleList(
    (0-5): 6 x Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.6, inplace=False)
      (4): Linear(in_features=1024, out_features=1024, bias=True)
      (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.6, inplace=False)
    )
  )
  (output_3d_pose_linear): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affi

Training args are: Namespace(action_loss_multiplier=0.005, batch_size=256, learning_rate=3e-05, num_epochs=30, pose_loss_multiplier=0.995, save_path='model_outputs\\simple_pose\\2024-11-13--16-12-40', testing_2d_data_path='datasets\\custom\\Processed\\test_2d_poses.npy', testing_3d_data_path='datasets\\custom\\Processed\\test_3d_poses.npy', testing_label_path='datasets\\custom\\Processed\\test_actions.npy', training_2d_data_path='datasets\\custom\\Processed\\train_2d_poses.npy', training_3d_data_path='datasets\\custom\\Processed\\train_3d_poses.npy', training_label_path='datasets\\custom\\Processed\\train_actions.npy')


INFO:root:Setup Optimizer
INFO:root:Setup Scheduler
INFO:root:Setup loss functions
INFO:root:Start Training and Testing Loops


Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 126/126 [00:01<00:00, 101.07it/s]


...

Saving at epoch 29
Epoch: 29 | Total Training Loss: 0.03872489184141159 | Pose Training Loss: 0.03751165047287941 | Action Training Loss: 0.2801634967327118 | Action Train Label Accuracy: 89.59433434801515


Testing =>: 100%|██████████| 10/10 [00:00<00:00, 318.14it/s]
INFO:root:Saving model current state


Epoch: 29 | Total Testing Loss: 0.033858295530080795 | Pose Testing Loss: 0.027694417163729668 | Action Testing Loss: 1.2604701519012451 | Action Test Label Accuracy: 54.125


<Figure size 640x480 with 0 Axes>

: 

<h2><u>5. Baseline 2 - SimplePoseGNN (Simple ML model using GNNs)</u></h2>

TODO.brian to write something meaningful here

In [ ]:
from src.simple_pose_gnn.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 3e-5,
    'num_epochs': 30,
    'batch_size': 256,
    'action_loss_multiplier': 0.005,
    'pose_loss_multiplier': 0.995,
    'training_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'custom', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_gnn', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

INFO:root:Model is currently using : cuda
INFO:root:Creating Missing Paths: model_outputs\simple_pose_gnn\2024-11-13--16-13-33
INFO:root:Saving config
INFO:root:Setup Training and Testing Dataloaders
INFO:root:Setup SimplePoseGNN model
INFO:root:SimplePoseGNN(
  (input_layer): Linear(in_features=2, out_features=80, bias=True)
  (pos_linear): Linear(in_features=5, out_features=80, bias=True)
  (blocks): ModuleList(
    (0-5): 6 x Sequential(
      (0): GraphConvModule(
        (conv_1): GraphConv(in=80, out=80, normalization=both, activation=None)
        (batch_norm_1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv_2): GraphConv(in=80, out=80, normalization=both, activation=None)
        (batch_norm_2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (feed_forward_1): Linear(in_features=80, out_features=80, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
      )
      (1): GraphConvModul

Training args are: Namespace(action_loss_multiplier=0.005, batch_size=256, learning_rate=3e-05, num_epochs=30, pose_loss_multiplier=0.995, save_path='model_outputs\\simple_pose_gnn\\2024-11-13--16-13-33', testing_2d_data_path='datasets\\custom\\Processed\\test_2d_poses.npy', testing_3d_data_path='datasets\\custom\\Processed\\test_3d_poses.npy', testing_label_path='datasets\\custom\\Processed\\test_actions.npy', training_2d_data_path='datasets\\custom\\Processed\\train_2d_poses.npy', training_3d_data_path='datasets\\custom\\Processed\\train_3d_poses.npy', training_label_path='datasets\\custom\\Processed\\train_actions.npy')
(32194, 13, 2)
(2400, 13, 2)
Starting EPOCH: 1 / 30


Training =>: 100%|██████████| 126/126 [00:48<00:00,  2.58it/s]


Saving at epoch 0
Epoch: 0 | Total Training Loss: 0.42331916093826294 | Pose Training Loss: 0.4173782765865326 | Action Training Loss: 1.605534553527832 | Action Train Label Accuracy: 24.054171584767346


Testing =>: 100%|██████████| 10/10 [00:30<00:00,  3.00s/it]
INFO:root:Saving model current state


...

Testing =>: 100%|██████████| 10/10 [00:27<00:00,  2.76s/it]
INFO:root:Saving model current state


Epoch: 18 | Total Testing Loss: 0.023667922243475914 | Pose Testing Loss: 0.01976788230240345 | Action Testing Loss: 0.799775242805481 | Action Test Label Accuracy: 65.66666666666666
Starting EPOCH: 20 / 30


Training =>: 100%|██████████| 126/126 [00:43<00:00,  2.86it/s]


Saving at epoch 19
Epoch: 19 | Total Training Loss: 0.04113205894827843 | Pose Training Loss: 0.03751901537179947 | Action Training Loss: 0.7601300477981567 | Action Train Label Accuracy: 65.2109088650059


Testing =>:   0%|          | 0/10 [00:00<?, ?it/s]

<h2><u>6. Improvement 1 - SimplePoseGAT</u></h2>

TODO.parash to write something meaningful here


In [ ]:
from src.simple_pose_gat.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 1e-4,
    'num_epochs': 20,
    'batch_size': 256,
    'action_loss_multiplier': 0.005,
    'pose_loss_multiplier': 0.995,
    'training_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'custom', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_gat', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)



<h2><u>7. Improvement 2 - SimplePoseTAG model</u></h2>

TODO.all

In [ ]:
from src.simple_pose_tag.train_and_test import training_loop
from argparse import Namespace
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

args_dict = {
    'learning_rate': 1e-4,
    'num_epochs': 20,
    'batch_size': 256,
    'action_loss_multiplier': 0.005,
    'pose_loss_multiplier': 0.995,
    'training_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_2d_poses.npy'),
    'training_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'train_3d_poses.npy'),
    'training_label_path': os.path.join('datasets', 'custom', 'Processed', 'train_actions.npy'),
    'testing_2d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_2d_poses.npy'),
    'testing_3d_data_path': os.path.join('datasets', 'custom', 'Processed', 'test_3d_poses.npy'),
    'testing_label_path': os.path.join('datasets', 'custom', 'Processed', 'test_actions.npy'),
    'save_path': os.path.join('model_outputs', 'simple_pose_gat', timestamp)
}
args = Namespace(**args_dict)
training_loop(args)

<h2><u>8. Evaluation & Analysis of models<u></h2>

<h2><u>9. Creating our own custom dataset</u></h2>

<h2><u>10. Evaluation on custom dataset</u></h2>

<h2><u>11. Conclusion</u></h2>

<h2><u>12. Video presentation & Resources</u></h2>